In [ ]:
# look into how to figure out what's in the df_mat
# (nested dictionary structure in matlab)
from scipy.io import loadmat
df_mat = loadmat('/home/disk/eos8/ach315/data/NASS_cropdata/all_crops_2014.mat')

In [92]:
import pandas as pd
import numpy as np

In [271]:
df = pd.DataFrame({'c1': [np.nan,20,np.nan,25,np.nan],
                   'c2': [1,2,3,np.nan,5],
                   'c3': [12,4,1,2,3]})

df

,c1,c2,c3
0,NaN,1.0,12
1,20.0,2.0,4
2,NaN,3.0,1
3,25.0,NaN,2
4,NaN,5.0,3


In [275]:
df.interpolate(method='linear')

,c1,c2,c3
0,NaN,1.0,12
1,20.0,2.0,4
2,22.5,3.0,1
3,25.0,4.0,2
4,25.0,5.0,3
